## Downloading datasets

In [3]:
!wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip
!unzip -q nature_12K.zip

--2025-04-19 04:04:38--  https://storage.googleapis.com/wandb_datasets/nature_12K.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.143.207, 173.194.69.207, 173.194.79.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.143.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3816687935 (3.6G) [application/zip]
Saving to: ‘nature_12K.zip’

nature_12K.zip      100%[===================>]   3.55G  40.2MB/s    in 93s     

2025-04-19 04:06:11 (39.3 MB/s) - ‘nature_12K.zip’ saved [3816687935/3816687935]



# PART B : Fine-tuning a Pre-trained Model

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader, random_split
import wandb

# === Fine-tuning Pretrained Model ===
def get_dataloaders(data_dir, batch_size=32):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    full_dataset = datasets.ImageFolder(root=data_dir, transform=transform)
    val_size = int(0.2 * len(full_dataset))
    train_size = len(full_dataset) - val_size
    train_set, val_set = random_split(full_dataset, [train_size, val_size])

    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=batch_size)

    return train_loader, val_loader

## Question 1: Loading and Modifying Pre-trained Model

## Explanation:

- Loads ResNet50 pre-trained on ImageNet

- Handles two key requirements:

  1. Image Size Compatibility: Uses standard 224x224 input size via transforms.Resize()

  2. Output Layer Modification: Replaces final layer (1000 classes) with new layer (10 classes)

- Provides three freezing strategies:

  - full: Freeze all layers except final

  - partial: Freeze layers until specified index

  - (implied none): No freezing, fine-tune all layers

In [11]:
def modify_model(num_classes=10, freeze_type="partial", freeze_until=6):
    model = models.resnet50(pretrained=True)

    if freeze_type == "full":
        for param in model.parameters():
            param.requires_grad = False
    elif freeze_type == "partial":
        for i, (name, param) in enumerate(model.named_parameters()):
            if i < freeze_until:
                param.requires_grad = False

    in_features = model.fc.in_features
    model.fc = nn.Linear(in_features, num_classes)
    return model

## Question 2: Fine-tuning Strategies

The code implements three strategies through freeze_type:

1. Full Freezing (freeze_type="full"):

  - All layers frozen except final classification layer

  - Only trains the new output layer

2. Partial Freezing (freeze_type="partial"):

  - Freezes layers up to freeze_until index

  - Fine-tunes later layers and final classifier

  - Common to freeze early feature extractors

3. No Freezing (implied when freeze_type is neither):

  - All layers are trainable

  - Complete fine-tuning of entire network

Key Trick: Using param.requires_grad = False to freeze layers and reduce computation

## Question 3: Fine-tuning Experiments

## Key Insights to Compare:

1. Training Speed: Frozen layers train faster

2. Accuracy: Partial freezing often gives best balance

3. Overfitting: Full freezing may underfit, no freezing may overfit

4. Resource Usage: More frozen layers = less memory/computation

In [12]:
def train_finetune(model, train_loader, val_loader, device, epochs=5, lr=1e-4):
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    # Only optimize parameters that require gradients
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)

    for epoch in range(epochs):
        model.train()
        total_loss, correct = 0.0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            correct += (outputs.argmax(1) == labels).sum().item()

        train_acc = correct / len(train_loader.dataset)

        # Validation
        model.eval()
        val_loss, val_correct = 0.0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                val_loss += criterion(outputs, labels).item()
                val_correct += (outputs.argmax(1) == labels).sum().item()

        val_acc = val_correct / len(val_loader.dataset)

        wandb.log({
            "train_loss": total_loss / len(train_loader),
            "train_acc": train_acc,
            "val_loss": val_loss / len(val_loader),
            "val_acc": val_acc,
            "epoch": epoch
        })

## Main Execution

In [13]:
def run():
    wandb.init(project="da6401-assignment2", config={
        "epochs": 5,
        "lr": 1e-4,
        "batch_size": 32,
        "freeze_type": "partial",  # full, partial, none
        "freeze_until": 6,
    })
    config = wandb.config

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    train_loader, val_loader = get_dataloaders("./inaturalist_12K/train", config.batch_size)

    model = modify_model(num_classes=10,
                         freeze_type=config.freeze_type,
                         freeze_until=config.freeze_until)

    train_finetune(model, train_loader, val_loader, device, config.epochs, config.lr)

if __name__ == "__main__":
    run()

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 91.5MB/s]


Epoch 1: Train Acc: 0.6841, Val Acc: 0.7674
Epoch 2: Train Acc: 0.8675, Val Acc: 0.7654
Epoch 3: Train Acc: 0.9374, Val Acc: 0.7579
Epoch 4: Train Acc: 0.9607, Val Acc: 0.7499
Epoch 5: Train Acc: 0.9633, Val Acc: 0.7514
